In [ ]:
import os
import json
import torch
import torch.optim as optim
from torchvision import transforms
from PIL import Image, ImageEnhance, ImageDraw
import numpy as np
from CRAFT.craft import CRAFT
import CRAFT.craft_utils
import CRAFT.imgproc
from lincenseplateocr.model import Model
from lincenseplateocr.utils import AttnLabelConverter, CTCLabelConverter
from lincenseplateocr.dataset import AlignCollate
from lincenseplateocr.test import validation


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Options:
    def __init__(self):
        self.exp_name = 'license_plate_training'  # 실험 이름
        self.train_data = 'lincenseplateocr/input/train'  # 트레이닝 데이터 경로
        self.valid_data = 'lincenseplateocr/input/vali'  # 검증 데이터 경로
        self.saved_model = 'lincenseplateocr/pretrained/Fine-Tuned.pth'  # STR 모델의 사전 학습된 모델
        self.num_iter = 3000  # 학습 반복 횟수
        self.valInterval = 200  # 검증 간격
        self.batch_size = 1  # 배치 크기
        self.lr = 0.001  # 학습률
        self.Prediction = 'CTC'  # STR의 예측 모드
        self.batch_max_length = 25  # 최대 라벨 길이
        self.imgH = 32  # 입력 이미지 높이
        self.imgW = 100  # 입력 이미지 너비
        self.character = '0123456789가나다라'  # 학습할 문자
        self.input_channel = 1  # 입력 채널 (흑백 이미지)
        self.output_channel = 512  # 출력 채널 수
        self.hidden_size = 256  # LSTM 히든 사이즈
        self.trained_craft_model = 'CRAFT/weights/craft_mlt_25k.pth'  # CRAFT 사전 학습된 가중치
        self.workers = 0  # 데이터 로더 워커 수

opt = Options()

# CRAFT 모델 로드
craft_net = CRAFT()
craft_net.load_state_dict(torch.load(opt.trained_craft_model))
craft_net = craft_net.to(device)
craft_net.eval()

# STR 모델 준비
converter = AttnLabelConverter(opt.character) if 'CTC' in opt.Prediction else CTCLabelConverter(opt.character)
opt.num_class = len(converter.character)
str_model = Model(opt).to(device)
str_model.load_state_dict(torch.load(opt.saved_model))

# 손실 함수 및 옵티마이저 설정
criterion = torch.nn.CTCLoss(zero_infinity=True).to(device)
optimizer = optim.Adam(str_model.parameters(), lr=opt.lr)


class DataAugmentation:
    """데이터 증강 클래스: 이미지 손상, 저해상도, 각도 조정, 이미지 일부 가리기"""
    
    def __init__(self, imgH, imgW):
        self.imgH = imgH
        self.imgW = imgW
        self.transform = transforms.Compose([
            transforms.Resize((imgH, imgW)),  # 기본 이미지 리사이즈
            transforms.RandomApply([self.add_noise()], p=0.3),  # 50% 확률로 노이즈 추가
            transforms.RandomApply([self.reduce_resolution()], p=0.3),  # 50% 확률로 해상도 저하
            transforms.RandomApply([transforms.RandomRotation(degrees=(-15, 15))], p=0.3),  # 50% 확률로 각도 조정
            transforms.RandomApply([self.occlude_image()], p=0.3),  # 50% 확률로 이미지 일부 가리기
            transforms.ToTensor()  # 텐서로 변환
        ])

    def add_noise(self):
        """이미지에 랜덤 노이즈 추가"""
        def noise(img):
            np_img = np.array(img)
            row, col, ch = np_img.shape
            mean = 0
            sigma = 10  # 노이즈 강도
            gauss = np.random.normal(mean, sigma, (row, col, ch))
            gauss = gauss.reshape(row, col, ch)
            noisy = np_img + gauss
            noisy = np.clip(noisy, 0, 255).astype(np.uint8)
            return Image.fromarray(noisy)
        return transforms.Lambda(noise)

    def reduce_resolution(self):
        """이미지의 해상도를 낮춤"""
        def low_res(img):
            # 현재 이미지 크기에서 다운샘플링 후 다시 업샘플링
            small_img = img.resize((self.imgW // 4, self.imgH // 4), Image.BILINEAR)
            return small_img.resize((self.imgW, self.imgH), Image.BILINEAR)
        return transforms.Lambda(low_res)

    def occlude_image(self):
        """이미지의 일부를 가림"""
        def occlude(img):
            draw = ImageDraw.Draw(img)
            # 이미지의 일부를 임의의 사각형으로 가림
            w, h = img.size
            x1, y1 = random.randint(0, w // 2), random.randint(0, h // 2)
            x2, y2 = random.randint(x1 + w // 4, w), random.randint(y1 + h // 4, h)
            draw.rectangle([x1, y1, x2, y2], fill=(0, 0, 0))  # 검정색으로 가림
            return img
        return transforms.Lambda(occlude)

    def __call__(self, img):
        """이미지에 변환 적용"""
        return self.transform(img)


def detect_text_craft(image, craft_net):
    """CRAFT로 텍스트 영역 탐지"""
    img_resized, target_ratio, _ = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        y, _ = craft_net(x)

    score_text = y[0, :, :, 0].cpu().data.numpy()
    score_link = y[0, :, :, 1].cpu().data.numpy()

    boxes, polys = craft_utils.getDetBoxes(score_text, score_link, text_threshold=0.7, link_threshold=0.4, low_text=0.4, poly=False)
    return boxes


def crop_text_regions(image, boxes):
    """탐지된 텍스트 영역을 자르기"""
    cropped_images = []
    for box in boxes:
        poly = np.array(box).astype(np.int32).reshape((-1, 2))
        x_min = np.min(poly[:, 0])
        y_min = np.min(poly[:, 1])
        x_max = np.max(poly[:, 0])
        y_max = np.max(poly[:, 1])

        cropped_img = image[y_min:y_max, x_min:x_max]
        cropped_images.append(cropped_img)

    return cropped_images


def load_label_json(label_path):
    """라벨 JSON 파일을 로드"""
    with open(label_path, 'r', encoding='utf-8') as f:
        label_data = json.load(f)
    return label_data['value']  # 이미지의 실제 라벨


def train(opt):
    """STR 모델 학습 루프"""
    data_augmentation = DataAugmentation(opt.imgH, opt.imgW)  # 데이터 증강

    for iteration in range(opt.num_iter):
        total_loss = 0
        # 학습용 데이터 폴더에서 파일 로드
        for img_file in os.listdir(opt.train_data):
            if img_file.endswith('.jpg'):  # 이미지 파일만 처리
                image_path = os.path.join(opt.train_data, img_file)
                label_path = os.path.splitext(image_path)[0] + ".json"

                # 이미지 및 라벨 로드
                image = imgproc.loadImage(image_path)
                label = load_label_json(label_path)

                # CRAFT로 텍스트 영역 탐지
                boxes = detect_text_craft(image, craft_net)
                cropped_images = crop_text_regions(image, boxes)

                # 각 자른 이미지 영역을 STR로 학습
                for cropped_image in cropped_images:
                    # 데이터 증강 적용
                    cropped_image_augmented = data_augmentation(cropped_image)

                    # 라벨 인코딩
                    text, length = converter.encode([label], batch_max_length=opt.batch_max_length)

                    # 모델 예측
                    preds = str_model(cropped_image_augmented.unsqueeze(0).to(device))
                    cost = criterion(preds, text)

                    # 손실 역전파
                    optimizer.zero_grad()
                    cost.backward()
                    optimizer.step()

                    total_loss += cost.item()

        if iteration % opt.valInterval == 0:
            print(f"Iteration {iteration}, Loss: {total_loss:.4f}")
            validate(opt)


def validate(opt):
    """검증 루틴"""
    AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=True)
    valid_dataset = TextRecognitionDataset(root=opt.valid_data, opt=opt)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=opt.batch_size, shuffle=True, num_workers=opt.workers, collate_fn=AlignCollate_valid)

    str_model.eval()
    with torch.no_grad():
        valid_loss, valid_accuracy = validation(str_model, criterion, valid_loader, converter, opt)
        print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.4f}")
    str_model.train()


# 트레이닝 실행
train(opt)
